# Fine-tuning for Sentiment Analysis
Mục tiêu tổng thể là tinh chỉnh (fine-tune) một mô hình ngôn ngữ đa dụng (distilbert-base-uncased) thành một mô hình phân loại cảm xúc chuyên biệt, sử dụng bộ dữ liệu tùy chỉnh với kích thước nhỏ (khoảng 100 bài đánh giá).

### **Bước cài đặt**
Sử dụng: !pip install --upgrade transformers datasets...

Mục tiêu: Cài đặt các thư viện cần thiết. Transformers cung cấp cho chúng ta mô hình tiền huấn luyện và Trainer (trình huấn luyện), datasets giúp quản lý dữ liệu văn bản, evaluate cho chúng ta chỉ số "accuracy" (độ chính xác) để đo lường kết quả.

### **Bộ dữ liệu (Dataset)**
Tạo ngẫu nhiên các danh sách Python chứa 100 bài đánh giá mẫu (50 tích cực, 50 tiêu cực).
Có thể sử dụng dataset tùy chỉnh, trong phạm vi bài làm không sử dụng lượng dataset lớn để tránh lãng phí thời gian.

### **Phân chia Train/Test**
Dùng dataset.train_test_split() để chia 100 bài đánh giá thành hai phần: 80 mẫu "tập huấn luyện" (training set) và 20 mẫu "tập kiểm thử" (test set).

Training set (80 mẫu): Mô hình được phép nhìn thấy và học hỏi từ dữ liệu này.

Test set (20 mẫu): Mô hình không bao giờ được thấy dữ liệu này trong quá trình huấn luyện. Chúng ta dùng nó ở cuối cùng để "chấm điểm" một cách trung thực, xem mô hình học cách tổng quát hóa tốt đến đâu, chứ không chỉ học vẹt. Được sử dụng ở trainer.evaluate() để so sánh dự đoán với đáp án thật (đã được gắn nhãn).

### **Tokenization (Mã hóa văn bản)**
ải một Tokenizer (bộ mã hóa) và dùng nó để chuyển đổi văn bản (ví dụ: "I love it") thành danh sách số (như [101, 1045, 2293, 2009, 102]).

Lý do: mô hình máy tính không hiểu được ngôn ngữ tự nhiên, trong đó tokenizer là "người phiên dịch" chuyển đổi các câu của con người sang định dạng số mà mô hình có thể xử lý.

### **Thiết lập Mô hình & Trình huấn luyện**
Tải mô hình distilbert, tạo TrainingArguments (tham số huấn luyện) và Trainer (trình huấn luyện).

Đối với mô hình: sử dụng mô hình distilbert, bằng cách tải nó với num_labels=2 và bản đồ id2label (ánh xạ ID sang nhãn), chúng ta có thể gắn "đầu ra phân loại cảm xúc" mới, chưa được huấn luyện (với nhãn "Positive" / "Negative") dựa theo nhu cầu.

TrainingArguments: Đây là đối tượng "cài đặt". Cho Trainer biết những điều cơ bản, như "huấn luyện trong 3 epoch (lượt)".

Trainer: gộp mô hình, cài đặt, và dữ liệu lại với nhau, phục vụ việc huấn luyện.

### **Huấn luyện (Training)**
Gọi trainer.train().

Mô tả: đây là bước "học hỏi". Trainer sẽ lặp qua 80 mẫu huấn luyện, yêu cầu mô hình đoán cảm xúc, kiểm tra với "đáp án", và "điều chỉnh" nhẹ các trọng số của để "đầu ra phân loại cảm xúc" trở nên chính xác hơn. Thực hiện trong 3 "epoch" (3 lần lặp qua toàn bộ 80 mẫu).

### **Đánh giá và Thử nghiệm**
Chạy trainer.evaluate() và sau đó sử dụng trainer.model trong một pipeline mới.

Quan sát và kiểm tra kết quả.

evaluate() dùng 20 mẫu kiểm thử mà mô hình chưa được thấy qua để đánh giá điểm số accuracy (độ chính xác) cuối cùng.

pipeline cho phép chúng ta sử dụng mô hình vừa được tinh chỉnh. Chứng minh rằng nó đã học và bây giờ đang tự đưa ra dự đoán ("Positive"/ "Negative") dựa trên 100 bài đánh giá đã tạo.




In [ ]:
#Force an upgrade to the latest versions
!pip install --upgrade transformers datasets accelerate evaluate scikit-learn -q

print("***Libraries installed.***")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
***Libraries installed.***


In [ ]:
import random

#1. Define templates and subjects
positive_templates = [
    "I absolutely loved the {}", "The {} was amazing", "{} is fantastic",
    "Highly recommend the {}", "{} was perfect", "What a wonderful {}",
    "I am so happy with the {}", "The {} exceeded my expectations",
    "This is the best {}", "10/10 for the {}"
]

negative_templates = [
    "I really hated the {}", "The {} was terrible", "{} is awful",
    "Would not recommend the {}", "{} was a complete disaster",
    "What a waste of time, the {} was bad", "I am so disappointed with the {}",
    "The {} failed to meet expectations", "This is the worst {}",
    "0/10 for the {}"
]

subjects = [
    "movie", "food", "service", "product", "experience", "book",
    "game", "show", "app", "hotel", "customer support", "delivery"
]

#2. Create the two lists
all_texts = []
all_labels = []

# Generate 50 positive reviews
for _ in range(50):
    template = random.choice(positive_templates)
    subject = random.choice(subjects)
    all_texts.append(template.format(subject))
    all_labels.append(1) # POSITIVE

# Generate 50 negative reviews
for _ in range(50):
    template = random.choice(negative_templates)
    subject = random.choice(subjects)
    all_texts.append(template.format(subject))
    all_labels.append(0) # NEGATIVE

#3. Shuffle the data
temp_list = list(zip(all_texts, all_labels))
random.shuffle(temp_list)
all_texts, all_labels = zip(*temp_list)

print(f"***Generated {len(all_texts)} reviews***")
print(f"Sample 1: '{all_texts[0]}' (Label: {all_labels[0]})")
print(f"Sample 2: '{all_texts[50]}' (Label: {all_labels[50]})")

***Generated 100 reviews***
Sample 1: 'The book exceeded my expectations' (Label: 1)
Sample 2: 'app is fantastic' (Label: 1)


In [ ]:
from datasets import Dataset

# 1. Create a single dataset from our lists
dataset = Dataset.from_dict({
    "text": all_texts,
    "label": all_labels
})

# 2. Shuffle and split the dataset
# test_size = 0.2 means 20% of the data goes to the test set
# shuffle=True is important to mix the positive/negative samples
dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=True)

# You now have two separate datasets:
# dataset_splits['train'] (48 samples)
# dataset_splits['test']  (12 samples)

print("***Dataset created and split***")
print(dataset_splits)

***Dataset created and split***
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 80
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 20
    })
})


In [ ]:
from transformers import AutoTokenizer

# We'll use 'distilbert-base-uncased' - it's small and fast
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    # This pads all sentences to the same length and truncates long ones
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply the tokenization to our entire dataset dictionary
tokenized_splits = dataset_splits.map(tokenize_function, batched=True)

print("***Both train and test sets are tokenized***")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

***Both train and test sets are tokenized***


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

#Load the model with friendly labels
model_name = "distilbert-base-uncased"
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# Load the model and add our labels
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

#Setup the accuracy metric
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    """This function calculates accuracy during evaluation."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#THE MINIMAL FIX
#We are using the simplest possible arguments to avoid the Colab bug.
print("--- Using minimal TrainingArguments to bypass the Colab error ---")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    #All other fancy arguments are removed to satisfy the old library.
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_splits["train"],
    eval_dataset=tokenized_splits["test"],
    compute_metrics=compute_metrics,
)

print("***Model and Trainer are ready***")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- Using minimal TrainingArguments to bypass the Colab error ---
***Model and Trainer are ready***


In [ ]:
print("***Starting training...***")

# This will train the model and evaluate it on the test set each epoch
trainer.train()

print("***Training finished!***")

***Starting training...***


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

In [ ]:
print("***Running final evaluation on the test set***")
eval_results = trainer.evaluate()

print("\n***Evaluation Results***")
print(f"Accuracy: {eval_results['eval_accuracy'] * 100:.2f}%")
print(f"Loss: {eval_results['eval_loss']:.4f}")

***Running final evaluation on the test set***



***Evaluation Results***
Accuracy: 100.00%
Loss: 0.2994


In [ ]:
from transformers import pipeline

print("***Testing the new fine-tuned model:***")

# We can pass the trainer's model and tokenizer directly
classifier = pipeline("sentiment-analysis", model=trainer.model, tokenizer=tokenizer)

#New positive sentence
text_to_test = "I really enjoyed my time here."
result = classifier(text_to_test)
print(f"Text: '{text_to_test}' -> Result: {result}")

#New negative sentence
text_to_test = "It was a complete letdown and I'm very angry."
result = classifier(text_to_test)
print(f"Text: '{text_to_test}' -> Result: {result}")

#None of our training samples (should be easy)
text_to_test = "I love my favorite streamer DoMixi"
result = classifier(text_to_test)
print(f"Text: '{text_to_test}' -> Result: {result}")

# Test 4: One of our test samples (should also be accurate)
# Note: The exact sample depends on the random shuffle.
test_sample_text = dataset_splits['test'][0]['text']
test_sample_label = "POSITIVE" if dataset_splits['test'][0]['label'] == 1 else "NEGATIVE"
result = classifier(test_sample_text)
print(f"\nTest Sample 0: '{test_sample_text}' (Actual: {test_sample_label})")
print(f"Result: {result}")

Device set to use cuda:0


***Testing the new fine-tuned model:***
Text: 'I really enjoyed my time here.' -> Result: [{'label': 'POSITIVE', 'score': 0.5781068801879883}]
Text: 'It was a complete letdown and I'm very angry.' -> Result: [{'label': 'NEGATIVE', 'score': 0.7461116313934326}]
Text: 'I love my favorite streamer DoMixi' -> Result: [{'label': 'POSITIVE', 'score': 0.6836006045341492}]

Test Sample 0: 'The delivery was amazing' (Actual: POSITIVE)
Result: [{'label': 'POSITIVE', 'score': 0.7104218006134033}]
